In [1]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd
import pickle
from sklearn import metrics, tree, svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold,cross_val_score,train_test_split,LeaveOneOut
from sklearn.naive_bayes import MultinomialNB
from statistics import mean

from data_load import *
from dictionaries import *
from score_models import *

import datetime
# a = datetime.datetime.now()
# time_list = []

In [2]:
column_list = [
                'problem_type',
                'creative',
                'outdoors',
                'career',
                'group_work',
                'liked_courses',
                'disliked_courses',
                'programming',
                'join_clubs',
                'not_clubs',
                'liked_projects',
                'disliked_projects',
                'tv_shows',
                'alternate_degree',
                'expensive_equipment',
                'drawing',
                'essay',
                'architecture',
                'automotive',
                'business',
                'construction',
                'health',
                'environment',
                'manufacturing',
                'technology',
                'program'
                ]

ohe_main =  [
            'problem_type',
            'creative',
            'outdoors',
            'career',
            'group_work',
            'liked_courses',
            'disliked_courses',
            'programming',
            'join_clubs',
            'not_clubs',
            'liked_projects',
            'disliked_projects',
            'tv_shows',
            'alternate_degree',
            'expensive_equipment',
            'drawing',
            'essay'
        ]

# Set the model experiment names and all of the used suffixes

### keep the suffixes and the experiment model names updated to view all scores

### It takes about 10 mins to score each experiment model name

### If a new type of encoding is being tried, changes need to be made to the scoring methods. Call Ayser.

In [3]:

experiment_model_names = [
#                         'd0_b0_c0_v0',
#                         'd0_b0_c1_v0',
#                         'd0_b0_c2_v0',
#                         'd0_b0_c3_v0',
#                         'd0_b1_c3_v0',
#                         'd0_b1_c0_v0',
#                         'd0_b1_c1_v0',
#                         'd0_b1_c2_v0',
#                         'd0_b0_c4_v0',
#                         'd0_b1_c4_v0',
#                         'd0_b0_c5_v0',
#                         'd0_b0_c29_v0',
#                         'd0_b1_c29_v0'
                        'd0_b1_c36_v0',
                        'd0_b0_c36_v0'
                        ]



In [4]:
multi_class_suffixes = [
                        'nb_le_f0_',
                        'nb_ohe_f0_',
                        'lrr_le_f0_',
                        'lrr_ohe_f0_',
                        'svm_le_f0_',
                        'svm_ohe_f0_'
                       ]

binary_class_suffixes = [
                        'nb_le_f1_',
                        'nb_ohe_f1_',
                        'lrr_le_f1_',
                        'lrr_ohe_f1_',
                        'svm_le_f1_',
                        'svm_ohe_f1_',
                        'tree_le_f1_',
                        'tree_ohe_f1_'
]

In [5]:
# This block does all of the scoring, last one to update, only cell that needs running
for experiment in experiment_model_names:
    scoring_dictionary = {}
    
    for mclass in multi_class_suffixes:
        temp_model_name = mclass+experiment
        model_name = temp_model_name
        
        model_data = pd.read_csv('exported_model_files/dataframes/'+model_name+'.csv',dtype=str)
        ohe = ohe_main
        # Loading test data
        if 'le' in model_name:
            test_data_t7 = get_label_encoded_data('data/t7.csv',model_name='t7',column_list=column_list,drop_not_happy='H',data_balance=False)[0]
        elif 'ohe' in model_name:
            test_data_t7 = get_merged_encoded_data(directory = 'data/t7.csv',model_name ='t7',one_hot_encode=ohe,column_list = column_list,drop_not_happy='H',data_balance=False)
       
        test_data_t7_temp = test_data_t7.copy()[list(model_data.columns)].head(210)

        # Loading model files
        pkl_file = open('exported_model_files/metadata/'+model_name+'_cat', 'rb')
        index_dict = pickle.load(pkl_file)
        new_vector = np.zeros(len(index_dict))

        pkl_file = open('exported_model_files/models/'+model_name+'.pkl', 'rb')
        model = pickle.load(pkl_file)

        # Preparing Loading data
        test_array = np.array(test_data_t7_temp.drop(axis=1,columns=["program"]))
        test_actual = np.array(test_data_t7_temp["program"])
        
        mclass_t3 = get_mclass_t3(temp_model_name,model,test_array,test_actual)
        mclass_RR = get_mclass_rr(temp_model_name,model,test_array,test_actual)
        mclass_accuracy = get_mclass_accuracy(temp_model_name,model,test_array,test_actual)
        mclass_reassignment = get_mclass_reassignment(temp_model_name,model)
        
        scoring_dictionary[mclass+experiment] = {'t3':mclass_t3,'RR':mclass_RR,'accuracy':mclass_accuracy,'reassignment':mclass_reassignment}
        print(str(temp_model_name)+" scored...")
        
#     for bclass in binary_class_suffixes:
#         temp_model_name = bclass+experiment
#         model_name = temp_model_name
        
#         model_data = pd.read_csv('exported_model_files/dataframes/'+model_name+'.csv',dtype=str)
#         ohe = ohe_main

#         # Loading test data
#         if 'le' in model_name:
#             test_data_t7 = get_label_encoded_data('data/t7.csv',model_name='t7',column_list=column_list,drop_not_happy='H',data_balance=False)[0]
#         elif 'ohe' in model_name:
#             test_data_t7 = get_merged_encoded_data(directory = 'data/t7.csv',model_name ='t7',one_hot_encode=ohe,column_list = column_list,drop_not_happy='H',data_balance=False)
            

#         test_data_t7_temp = test_data_t7.copy()[list(model_data.columns)].head(210)
 
#         # Getting average accuracy score
#         test_array = np.array(test_data_t7_temp.drop(axis=1,columns=["program"]))
#         test_actual = np.array(test_data_t7_temp["program"])
        
#         bclass_t3 = get_bclass_t3(test_array,model_name,test_actual,test_data_t7)
#         bclass_RR = get_bclass_rr(test_array,model_name,test_actual,test_data_t7)
#         bclass_accuracy = get_bclass_accuracy(test_array,model_name,test_actual,test_data_t7)
#         bclass_reassignment = get_bclass_reassignment(test_array,model_name,test_data_t7)
#         scoring_dictionary[bclass+experiment] = {'t3':bclass_t3,'RR':bclass_RR,'accuracy':bclass_accuracy,'reassignment':bclass_reassignment}
#         print(str(temp_model_name)+" scored...")
        
    print("saving "+str(experiment)+ " scores..")
    save_scores(scoring_dictionary,experiment)
    b = datetime.datetime.now()
    c = b - a
    a = datetime.datetime.now()
    time_list.append(c)
    
    
print("scoring complete.")
    

nb_le_f0_d0_b1_c36_v0 scored...
nb_ohe_f0_d0_b1_c36_v0 scored...
lrr_le_f0_d0_b1_c36_v0 scored...
lrr_ohe_f0_d0_b1_c36_v0 scored...
svm_le_f0_d0_b1_c36_v0 scored...
svm_ohe_f0_d0_b1_c36_v0 scored...
saving d0_b1_c36_v0 scores..
nb_le_f0_d0_b0_c36_v0 scored...
nb_ohe_f0_d0_b0_c36_v0 scored...
lrr_le_f0_d0_b0_c36_v0 scored...
lrr_ohe_f0_d0_b0_c36_v0 scored...
svm_le_f0_d0_b0_c36_v0 scored...
svm_ohe_f0_d0_b0_c36_v0 scored...
saving d0_b0_c36_v0 scores..
scoring complete.


In [6]:
for c in time_list:
    print(str(c.seconds)+" seconds or "+str(c.seconds/60)+" mins")


3 seconds or 0.05 mins
3 seconds or 0.05 mins


# Delete:

In [35]:
new_vector = np.zeros(21)
new_vector[0] =  '1'
new_vector[1] =  '1'
new_vector[2] =  '1'
new_vector[3] =  '1'
new_vector[4] =  '1'
new_vector[5] =  '1'
# new_vector[7] =  0 #programming[post_dict['programming'][0]]
new_vector[6] =  '1'
new_vector[7] =  '1'
new_vector[8] = '1'
new_vector[9] = '1'
# new_vector[12] = 0 #tv_shows[post_dict['tv_shows'][0]]
new_vector[10] = '1'
# new_vector[14] = 0 #expensive_equipment[post_dict['expensive_equipment'][0]]
new_vector[11] = '1'
new_vector[12] = '1'
new_vector[13] = '1'
new_vector[14] = '1'
new_vector[15] = '1'
new_vector[16] = '1'
new_vector[17] = '1'
new_vector[18] = '1'
new_vector[19] = '1'
new_vector[20] = '1'

print(new_vector)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [36]:
new_vector = list(new_vector)
for i in range(len(new_vector)):
    new_vector[i]  = str(int(new_vector[i]))

columns = [
            'creative',
            'outdoors',
            'career',
            'group_work',
            'liked_courses',
            'disliked_courses',
            'join_clubs',
            'not_clubs',
            'liked_projects',
            'disliked_projects',
            'alternate_degree',
            'drawing',
            'essay',
            'architecture',
            'automotive',
            'business',
            'construction',
            'health',
            'environment',
            'manufacturing',
            'technology'
]
t7 = get_label_encoded_data('data/t7.csv',model_name='t7',column_list=columns,drop_not_happy='H',data_balance=False)[0]
data_to_append = {}
for i in range(len(columns)):
    data_to_append[t7.columns[i]] = int(new_vector[i])
t7 = t7.append(data_to_append, ignore_index = True)
t7 = pd.get_dummies(t7,columns=columns)
rename_columns = {
    'architecture_1':'architecture',
    'automotive_1':'automotive',
    'business_1':'business',
    'construction_1':'construction',
    'health_1':'health',
    'environment_1':'environment',
    'manufacturing_1':'manufacturing',
    'technology_1':'technology'
}
drop_columns = [
    'architecture_0',
    'automotive_0',
    'business_0',
    'construction_0',
    'health_0',
    'environment_0',
    'manufacturing_0',
    'technology_0'
]
t7 = t7.rename(index=str,columns = rename_columns)
t7 = t7.drop(drop_columns, axis=1)

print(t7.columns)
new_vector = np.array(t7[len(t7)-1:len(t7)])


getting label encoded data...
Retrieved label encodede data...
encoding columns
writing columns...
returning label encoded data...
Index(['creative_0', 'creative_1', 'creative_2', 'outdoors_0', 'outdoors_1',
       'outdoors_2', 'career_0', 'career_1', 'career_2', 'career_3',
       'career_4', 'career_5', 'career_6', 'group_work_0', 'group_work_1',
       'group_work_2', 'liked_courses_0', 'liked_courses_1', 'liked_courses_2',
       'liked_courses_3', 'liked_courses_4', 'liked_courses_5',
       'liked_courses_6', 'liked_courses_7', 'liked_courses_8',
       'liked_courses_9', 'liked_courses_10', 'disliked_courses_0',
       'disliked_courses_1', 'disliked_courses_2', 'disliked_courses_3',
       'disliked_courses_4', 'disliked_courses_5', 'disliked_courses_6',
       'disliked_courses_7', 'disliked_courses_8', 'disliked_courses_9',
       'disliked_courses_10', 'join_clubs_0', 'join_clubs_1', 'join_clubs_2',
       'join_clubs_3', 'join_clubs_4', 'join_clubs_5', 'join_clubs_6',
    

In [37]:
model_name = 'nb_ohe_f0_d0_b0_c36_v0'
pkl_file = open('exported_model_files/models/'+model_name+'.pkl', 'rb')
model = pickle.load(pkl_file)

In [38]:
prediction = model.predict_proba([new_vector[0])